In [1]:
# universally modules
import re
import sys
import regex
import nltk.data
import numpy as np
import pandas as pd
from tqdm import tqdm
sys.path.append("../src")

# preprocessing and transformation modules
import fasttext
import Preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# model algorithm
from xgboost import XGBRegressor, XGBClassifier

# evaluation modules
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import classification_report

/home/bigdata/anaconda3/lib/python3.8/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.5) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.5). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


### Parameters

In [2]:
# to speed up the process choose a sample size to randomly draw a sample of the whole daataset
sample_size = 100_000

# remove all text that contain less than n chars
min_chars_per_text = 50

# which features will be used for the TF-IDF transformation
text_features = "text_preprocessed"

#### define the target variable and categorial variables used in later transformations ###
#### Case 1: gender 
target_variable = "gender"
categorial_variables =  ["topic", "sign"]

# Case 2: topic
#target_variable = "topic"
#categorial_variables =  ["gender", "sign"]

# Case 3: age
#target_variable = "age"
#categorial_variables =  ["topic", "gender", "sign"]

# Case 4: sign
#target_variable = "sign"
#categorial_variables =  ["gender", "topic"]
############################################################################################

# use only words that occur at least sqrt_3(X) times 
min_df_exponent = (1/3)

### Initialization 

In [3]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
df = pd.read_csv("../resource/data/blogtext.csv")

In [4]:
# draw random sample for faster processing:
df = df.sample(sample_size)
df

,id,gender,age,topic,sign,date,text
493508,2114453,female,24,Law,Cancer,"22,April,2004",From the Sublime to the Ridiculous...
364627,3509949,female,23,Education,Leo,"07,July,2004","Imagine,if you will, 700 sleep-deprived..."
312781,581839,female,16,Student,Taurus,"02,August,2004",alrighty then... my sister has been...
268110,1325355,female,26,indUnk,Cancer,"28,June,2004","Diana, Happy belated birthday! It soun..."
207575,1788812,male,25,Consulting,Taurus,"08,August,2004",urlLink It's me!&nbsp; urlLink ...
...,...,...,...,...,...,...,...
458296,518116,female,27,indUnk,Pisces,"12,June,2002",ok...see 6/3/02...started to think I ha...
369006,1609589,male,16,Student,Capricorn,"27,May,2004","Dustin said I should update, so I am- I..."
68226,320176,male,45,Publishing,Scorpio,"16,May,2004",From the story it sounds like he ...
255722,106651,male,25,indUnk,Leo,"01,June,2004",The title of the page reads ' url...


### Filtering

In [5]:
# filter for a mininmal number of letters in a tweet:
df = df[df["text"].str.count(r"[a-zA-Z]") >= min_chars_per_text]
df = df.reset_index(drop=True)

### Feature Engineering

In [6]:
#def findDates(text):
#    try:
#        return len([date for date in\
#                    datefinder.find_dates(text)])
#    except:
#        return 0

In [7]:
def buildFeatures(text):
    text_split = text.split()
    len_text = len(text)
    sentence_split = tokenizer.tokenize(text)
    
    # find the number of urls in the text
    keywords = ["urlLink","http","www"]
    nb_urls = sum((any(keyword in pattern for keyword in keywords))\
               for pattern in text.split())
    # find the number of mails in the text
    nb_mails = len(re.findall(r"([a-zA-Z0-9+._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+|\bmail\b)"\
                      ,text))
    
    # find the number of dates in the text
#    nb_dates = findDates(text)
     
    # find characteristics about the usage of letters, numbers and symbols
    uppercase_ratio = len(re.findall(r'[A-Z]', text))/len_text
    lowercase_ratio = len(re.findall(r'[a-z]', text))/len_text
    number_ratio = len(re.findall(r'[0-9]', text))/len_text
    symbol_ratio = len(re.findall(r'[$-/:-?{-~!"^_`\[\]]', text))/len_text

    # find characteristics about the letters per word
    sentence_len_word = [len(word) for word in text_split]
    avg_letters_per_word = np.mean([len(word) for word in text_split])
    var_letters_per_word = np.var([len(word) for word in text_split])
    unique_words_ratio = len(set(text_split))/len(text_split)

    # find characteristics about the letters per sentence
    sentence_len_list = [len(sentence) for sentence in sentence_split]
    avg_letters_per_sentence = np.mean(sentence_len_list)
    var_letters_per_sentence = np.var(sentence_len_list)
    
    # find characteristics about the words per sentence
    words_per_sentence_len_list = [len(sentence.split()) for sentence in sentence_split]
    avg_words_per_sentence = np.mean(words_per_sentence_len_list)
    var_words_per_sentence = np.var(words_per_sentence_len_list)
    
    # find the trumps
    uppercase_per_sentence_ratio = [len(re.findall(r'[A-Z]', sentence))/len(sentence)\
                                    for sentence in sentence_split]
    max_sentence_uppercase_ratio = max(uppercase_per_sentence_ratio)
    max_sentence_uppercase_len = len(sentence_split[uppercase_per_sentence_ratio.index(max_sentence_uppercase_ratio)])
    
    return len_text, nb_urls, nb_mails,\
           uppercase_ratio, lowercase_ratio, number_ratio, symbol_ratio,\
           avg_letters_per_word, var_letters_per_word, unique_words_ratio,\
           avg_letters_per_sentence, var_letters_per_sentence,\
           avg_words_per_sentence, var_words_per_sentence,\
           max_sentence_uppercase_ratio, max_sentence_uppercase_len        
           

In [8]:
# append the data
features = [buildFeatures(text) for text  in tqdm(df["text"])]

# append the data
columns = ["Text length", "Number URLs", "Number mails",\
          "Uppercase ratio", "Lowercase ratio", "Number ratio", "Symbol ratio",\
          "Average letters per word", "Variance of letters per word", "Unique words ratio",\
          "Average letters per sentence", "Variance of letters per sentence",\
          "Average words per sentence", "Variance of words per sentence",\
          "Maximal uppercase ratio per sentence", "Length of the maximal uppercase ratio sentence"]

# merge the features with the original dataset
df_preprocessed = df.merge(pd.DataFrame(features, columns=columns), left_index=True, right_index=True)

100%|██████████| 88/88 [00:00<00:00, 702.79it/s]


### Text Preprocessing

In [9]:
# use the preprocessing  module
preprocessor = Preprocessing.Preprocessing()
df_preprocessed["text_preprocessed"] = preprocessor.ProcessMany(df_preprocessed["text"])

# predict the main language
model = fasttext.load_model('../src/data/lid.176.ftz')
df_preprocessed["main_language"] = [model.predict(text)[0][0].split("__")[-1] for text in tqdm(df_preprocessed["text_preprocessed"])]

100%|██████████| 88/88 [00:00<00:00, 4712.52it/s]


In [10]:
def remove_non_ascii_words(text):
    return "".join([f"{word} " for word in text.split() if len(re.findall(r'[^\x00-\x7f]', word)) == 0])


In [11]:
# drop unnecassary features
df_filtered = df_preprocessed[(df_preprocessed["main_language"] == "en")]\
                .drop(["id","text","date","main_language"], axis= 1)

# remove all words with non-ascii chars
df_filtered["text_preprocessed"] = df_filtered["text_preprocessed"].apply(remove_non_ascii_words)

### Transformation

In [12]:
class StackedTransformation:
    
    def  __init__(self, X=None, y=None, numerical_transformer=None, text_transformer=None, text_features=None):
        self.X = X
        self.y = y
        self.numerical_transformer = numerical_transformer
        self.text_transformer = text_transformer
        self.text_features = text_features
        
    def data_split(self, X, y, test_size= 0.2):
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=test_size, random_state=42)
        return self.X_train, self.X_test, self.y_train, self.y_test
        
    def build_transformer(self, X=None, y=None,\
                          numerical_transformer=None, text_transformer=None, text_features=None):
        # check if variables are present
        if X == None:
            X = self.X
        if y == None:
            y = self.y
        if numerical_transformer == None:
            numerical_transformer = self.numerical_transformer
        if text_transformer == None:
            text_transformer = self.text_transformer
        if text_features == None:
            text_features = self.text_features

        # split the data
        
        self.data_split(X, y)
        
        # Create datasets for each classifier
        self.X_train_text = self.X_train[text_features]
        self.X_test_text =  self.X_test[text_features]
        
        self.X_train_numerical = self.X_train.drop(text_features, axis=1)
        self.X_test_numerical = self.X_test.drop(text_features, axis=1)

        # create transformers 
        self.numerical_transformer = numerical_transformer
        self.text_transformer = text_transformer
        
        #create transformed training batches
        self.X_train_numerical_transformed = self.numerical_transformer.fit_transform(self.X_train_numerical)
        self.X_test_numerical_transformed = self.numerical_transformer.transform(self.X_test_numerical)

        self.X_train_text_transformed = self.text_transformer.fit_transform(self.X_train_text)
        self.X_test_text_transformed = self.text_transformer.transform(self.X_test_text)

    def transform_one(self, text):
        entry = pd.DataFrame(columns=text.keys())
        entry = entry.append(text, ignore_index=True)
        
        # create datasets for each transformer
        entry_text = entry[text_features]
        entry_numerical = entry.drop(text_features, axis=1)
        
        entry_numerical_transformed = self.numerical_transformer.transform(entry_numerical)
        entry_text_transformed = self.text_transformer.transform(entry_text)
        
        return {"transformed_text": entry_text_transformed,\
               "transformed_numerical": entry_numerical_transformed}


### Data Split

In [13]:
X,y = df_filtered.drop(target_variable, axis=1),df_filtered[target_variable]

In [14]:
# use the  text transformer class to create two transformers for the textual and the numerical model
text_transformer = TfidfVectorizer(ngram_range=(1,1), min_df=int(len(X)**(min_df_exponent)))
numerical_transformer = make_column_transformer((OneHotEncoder(handle_unknown="ignore"), categorial_variables)\
                                                       , remainder=StandardScaler())

stacking = StackedTransformation(X, y, numerical_transformer, text_transformer, text_features)
stacking.build_transformer()

### Clustering

In [15]:
text_features = stacking.text_transformer.get_feature_names()
text_data = stacking.X_train_text_transformed.toarray()

df_text_cluster = pd.DataFrame(text_data, columns=text_features)
df_text_cluster

# Die Features beschreiben die Worte im Text
# Die Werte sind die TF*IDF-transformierten Textdaten

,11,15,200,act,action,add,age,ago,anymore,apparently,...,word,work,world,worry,write,wrong,ya,yeah,year,yesterday
0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.0,0.167960,0.0,0.0,0.000000,0.00000,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.123859,0.000000,0.159545,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,0.145061,0.166575,0.0,0.000000,0.0,0.0,0.000000,0.13967,0.0,0.0,...,0.079115,0.103119,0.000000,0.0,0.000000,0.067423,0.000000,0.000000,0.105466,0.000000
65,0.102620,0.000000,0.0,0.117839,0.0,0.0,0.000000,0.00000,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.190788,0.000000,0.000000,0.000000,0.000000
66,0.000000,0.000000,0.0,0.115092,0.0,0.0,0.057546,0.00000,0.0,0.0,...,0.054663,0.000000,0.052225,0.0,0.000000,0.093170,0.000000,0.000000,0.036435,0.000000
67,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,...,0.173318,0.225903,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.115523,0.182459


In [16]:
numerical_data = stacking.X_train_numerical_transformed
numerical_features = np.append(stacking.numerical_transformer.transformers_[0][1].get_feature_names(),\
                     stacking.X_train_numerical.columns.drop(categorial_variables))

df_numerical_cluster = pd.DataFrame(numerical_data, columns=numerical_features)
df_numerical_cluster


# Die Features mit den x0 - xi Werten beschreiben die Ausprägungen die kategorialen Variablen
# das jeweilige i beschreibt das i-te Element der im Punkt "Target Variable" definierten liste categorial_variables
# Die verbleibenden Features (ohne xi) sind Standardskaliert, (x - \mu)/\sigma

,x0_Agriculture,x0_Architecture,x0_Arts,x0_Communications-Media,x0_Consulting,x0_Education,x0_Engineering,x0_HumanResources,x0_Internet,x0_Law,...,Symbol ratio,Average letters per word,Variance of letters per word,Unique words ratio,Average letters per sentence,Variance of letters per sentence,Average words per sentence,Variance of words per sentence,Maximal uppercase ratio per sentence,Length of the maximal uppercase ratio sentence
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.188966,-0.211855,-0.141969,0.249935,-0.109786,-0.209599,-0.100388,-0.235177,-0.258180,0.348741
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.760447,-0.032599,-0.199648,0.076483,-0.321340,-0.309126,-0.327829,-0.285257,-0.321831,-0.333540
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.699487,-0.251692,-0.255734,1.601238,0.097803,-0.399956,0.048626,-0.353575,-0.788451,0.262183
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.733598,-0.806096,-0.252732,0.099943,-0.421116,-0.374826,-0.407464,-0.328854,-0.852262,-0.175699
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.655676,0.026087,-0.126142,1.362211,0.977000,-0.399956,0.958393,-0.353575,-0.694032,1.087031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.536923,0.602167,0.203957,-0.927789,1.000337,5.946264,0.917610,3.613456,0.562219,-0.384457
65,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.808169,-0.100501,0.070874,-1.263148,-0.236435,-0.306272,-0.231552,-0.270539,-0.466494,-0.134966
66,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.028254,-0.417425,-0.146809,-1.318120,-0.382460,-0.310825,-0.373750,-0.273281,-0.145021,-0.353907
67,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.454660,-0.023920,-0.147297,-0.392117,-0.165573,-0.129866,-0.163592,-0.145085,-0.278824,-0.226616
